<a href="https://colab.research.google.com/github/nechebarrena/HOML/blob/main/HOML_cap_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CAPITULO 8 - HANDS ON MACHINE LEARNING**
## **Reduccion de dimensionalidad**

Existen muchos problemas donde contamos con enormes cantidades de atributos. En general, tener muchos atributos, vuelve mas complejo al problema y uno mucha veces necesita simplificarlo. Una forma de hacer esto es reducir la cantidad de atributos que nuestro modelo va a procesar, para eso existen diversas tecnicas de reduccion de la dimensionalidad. Ademas, estas mismas tecnicas muchas veces se utilizan con el objetivo de ayudar a la visualizacion de ciertos aspectos del problema. Para estos dos objetivos el libro propone tratar las tres tecnicas princiapes, PCA, Kernel PCA, y LLE.



El principal problema de los sistemas con un gran numero de dimensiones es que conforme aumenta la cantidad de estas, aumenta la necesidad de datos de entrenamiento para llenar el espacio de fases. Esto se debe a que, para simplificar, al aumentar la cantidad de dimensiones aumenta el volumen del espacio y por lo tanto la cantidad de puntos que lo llenan. Esto a su vez tiene otra consecuencia un poco menos intuitiva, cuanto mas grande es la cantidad de dimensiones los puntos tienden a concentrarse mas sobre la superficie del volumen. Esto significa que en general la distancia entre 2 puntos tiende a ser mayor y por lo tanto, cuando entrenemos un modelo que haga una interpolacion entre ambos sera menos preciso. 

### **Metodos principales** 

Existen 2 metodos que son los principales a la hora de implementar una reduccion de dimensionalidad que vamos a ver a continuacion:

#### **Proyeccion**

Muchas veces los datos experimentales con los que contamos pueden ser proyectados en sub espacios del espacio original. Esta proyeccion implica perder alguna dimension original del problema, o sea, perder informacion. Sin embargo, uno puede buscar cual es el sub espacio que permite reducir la dimension minimizando la perdida de informacion. Para eso debemos buscar cual es este sub espacio "que se parece" a los datos.

En las figuras $8.2$ y $8.3$ del libro se muestra un ejemplo de esto que deja bastante clara esta situacion y lo que se busca.

#### **Aprendizaje con Manifolds**

Un _d-manifold_ es una superficie "enrollada" en un espacio de _d_ dimensiones dentro de un espacio mayor de _n_ dimensiones. Esto que suena dificil de explicar es muy facil de ver. En la figura $8.4$ del libro se muestra un ejemplo muy claro. Se puede ver una nube de puntos en _3-D_ que sin embargo puede ser "desenrrollada" en un plano _2-D_.

La hipotesis detras de estos metodos consta en general de 2 partes. La primera es la que ya mencionamos, que los datos pueden expresarse en un espacion menor con una perdida muy chica de informacion. La segunda parte de la hipotesis es que uno luego puede realizar una tarea posterior (clasificacion, regresion, etc) sobre este nuevo espacio de forma mas sencilla. Esta segunda hipotesis no siempre es cierta. La figura $8.6$ del libro ejemplifica esto con un conjunto de datos donde una tarea de clasificacion resulta mas "sencilla" sobre el espacio original que sobre el espacio reducido.

Ahora que vimos por arriba la idea general de la reduccion de dimensionalidad veamos algunos de los metodos las utilizados.



#### **PCA**
